In [1]:
import time
import numpy as np
import visit
visit.Launch()

True

In [2]:
field_nulls = np.loadtxt('../../centroid_fitting/output/2016-07-19/field_nulls.txt')

In [3]:
print field_nulls

[[-0.00478808 -0.00640996]
 [ 0.00389906 -0.01285931]
 [ 0.00880345 -0.0124718 ]
 [ 0.01678544 -0.01155452]
 [ 0.02348968 -0.00646158]
 [ 0.02656863  0.00363246]
 [ 0.02842711  0.0115303 ]
 [ 0.02566465  0.01639859]
 [ 0.02455535  0.02170641]
 [ 0.02405411  0.02773693]
 [ 0.02238233  0.03156278]
 [ 0.01939845  0.0334338 ]
 [ 0.00851801  0.0394233 ]
 [ 0.00167052  0.04778751]
 [ 0.00113474  0.03081366]
 [-0.0117485   0.02482263]
 [-0.01249873  0.02285998]
 [-0.01660147  0.01428564]
 [-0.01584593  0.01120546]
 [-0.01061126 -0.00568984]
 [-0.00593098 -0.00578328]]


In [4]:
def launch_points(center):
    thetas = np.linspace(0, 3./4.*np.pi, 60)
    thetas = np.concatenate((thetas, np.linspace(5./4.*np.pi, 2.*np.pi, 60)))
    x_outer = 0.01 * np.cos(thetas) + center[0]
    y_outer = 0.01 * np.sin(thetas) + center[1]
    z_outer = 0.249 * np.ones((x_outer.size))
    points_outer = np.empty((x_outer.size + y_outer.size + z_outer.size))
    points_outer[0::3] = x_outer
    points_outer[1::3] = y_outer
    points_outer[2::3] = z_outer
    points_outer = tuple(points_outer)

    thetas = np.linspace(0, 2.*np.pi, 80)
    x_inner = 0.005 * np.cos(thetas) + center[0]
    y_inner = 0.005 * np.sin(thetas) + center[1]
    z_inner = 0.249 * np.ones(x_inner.size)
    points_inner = np.empty((x_inner.size + y_inner.size + z_inner.size))
    points_inner[0::3] = x_inner
    points_inner[1::3] = y_inner 
    points_inner[2::3] = z_inner
    points_inner = tuple(points_inner)
    
    return points_outer, points_inner

points_outer, points_inner = launch_points(field_nulls[0])

In [5]:
visit.OpenDatabase('/Users/vonderlinden2/rsx_analysis/writing_to_vtk/output/2016-07-06/all*.vts database')

visit.AddPlot("Pseudocolor", "Te", 1, 0)
PseudocolorAtts = visit.PseudocolorAttributes()
PseudocolorAtts.scaling = PseudocolorAtts.Linear  # Linear, Log, Skew
PseudocolorAtts.limitsMode = PseudocolorAtts.OriginalData  # OriginalData, CurrentPlot
PseudocolorAtts.colorTableName = "Purples"
visit.SetPlotOptions(PseudocolorAtts)
visit.AddOperator("Slice", 0)
SliceAtts = visit.SliceAttributes()
SliceAtts.originType = SliceAtts.Intercept  # Point, Intercept, Percent, Zone, Node
SliceAtts.originIntercept = 0.249
SliceAtts.axisType = SliceAtts.ZAxis
SliceAtts.project2d = 0
visit.SetOperatorOptions(SliceAtts, 0)

visit.AddPlot("Streamline", "B", 1, 0)
StreamlineAtts_outer = visit.StreamlineAttributes()
StreamlineAtts_outer.sourceType = StreamlineAtts_outer.SpecifiedPointList
StreamlineAtts_outer.SetPointList(points_outer)
StreamlineAtts_outer.radius = 0.01
StreamlineAtts_outer.sampleDensity0 = 50
StreamlineAtts_outer.fillInterior = 0
StreamlineAtts_outer.coloringMethod = StreamlineAtts_outer.Solid  # Solid, ColorBySpeed, ColorByVorticity, ColorByLength, ColorByTime, ColorBySeedPointID, ColorByVariable, ColorByCorrelationDistance, ColorByNumberDomainsVisited
StreamlineAtts_outer.colorTableName = "Default"
StreamlineAtts_outer.singleColor = (255, 0, 0, 255)
visit.SetPlotOptions(StreamlineAtts_outer)

visit.AddPlot("Streamline", "B", 1, 0)
StreamlineAtts_inner = visit.StreamlineAttributes()
StreamlineAtts_inner.sourceType = StreamlineAtts_inner.SpecifiedPointList
StreamlineAtts_inner.SetPointList(points_inner)
StreamlineAtts_inner.radius = 0.005
StreamlineAtts_inner.sampleDensity0 = 50
StreamlineAtts_inner.fillInterior = 0
StreamlineAtts_inner.coloringMethod = StreamlineAtts_inner.Solid  # Solid, ColorBySpeed, ColorByVorticity, ColorByLength, ColorByTime, ColorBySeedPointID, ColorByVariable, ColorByCorrelationDistance, ColorByNumberDomainsVisited
StreamlineAtts_inner.colorTableName = "Default"
StreamlineAtts_inner.singleColor = (190, 64, 0, 255)
visit.SetPlotOptions(StreamlineAtts_inner)

visit.DrawPlots()

1L

In [8]:
date = '2016-07-20'
output_path = '../output/' + date + '/electron_canonical_flux_tube'
ending = '.png'

save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)

for time_point in xrange(visit.TimeSliderGetNStates()):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = launch_points(field_nulls[time_point])
    
    visit.SetActivePlots(1)
    StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [9]:
!ffmpeg -framerate 1/2 -i ../output/2016-07-20/electron_canonical_flux_tube%04d.png -r 1/2 -pix_fmt yuv420p \
 -vcodec libx264 -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -y ../output/2016-07-20/out.mp4

ffmpeg version 2.8.1 Copyright (c) 2000-2015 the FFmpeg developers
  built with Apple LLVM version 6.0 (clang-600.0.56) (based on LLVM 3.5svn)
  configuration: --prefix=/Users/vonderlinden2/homebrew/Cellar/ffmpeg/2.8.1_1 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libvo-aacenc --enable-libxvid --enable-vda
  libavutil      54. 31.100 / 54. 31.100
  libavcodec     56. 60.100 / 56. 60.100
  libavformat    56. 40.101 / 56. 40.101
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 40.101 /  5. 40.101
  libavresample   2.  1.  0 /  2.  1.  0
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.101 /  1.  2.101
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from '../output/2016-07-20/electron_canonical_flux_tube%04d.png':
  Duration: 00:00:42.00, start: 0.000000, bitrate: N/A
    Stream #0

In [18]:
pwd

u'/Users/vonderlinden2/rsx_analysis/canonical_flux_tubes/source'

In [10]:
visit.SaveWindow?